In [ ]:
import io

import numpy as np
import pandas as pd

この講義では、「データフレーム」を使って、データを**キレイに(tidy)** 表現する方法を説明します。
本講義では データフレームのためのライブラリとして`pandas` を使用します。

## CVS形式とは (What is CSV format)

表のようなデータを表現できる方法は複数がありますが、プログラムでデータを扱うのために特に使いやすいのはCSV形式です。
CSV形式は、プログラムによって生成または手動の生成、両方とも簡単で、読み込みも簡単にできます。
CSVはComma-separated valuesの略で、カンマ区切りという意味です。

CSV形式のルールは以下です。

* 各行はカンマで区切っているいくつかの値から成り立っています。一つの値はフィールドといいます。
* 各行はフィールドの数は同じです。　一行はレコードといいます。
* 値のなかではカンマ、改行、引用符は原則として入りません。
* もしカンマ、改行を入れなければいけない場合、引用符の中に入れます： `"a,b"`
  * 引用符を入れなければいけない場合は、引用符の中に二重しなければなりません： `"a""b"`
* ファイルの最初の一行はヘッダ行を入れることができます。必須ではありませんが、できればあった方がいいです。

普段はCSV形式`.csv`のファイルとして保存しますが、Pythonでは直接のプログラムへの組み込みも可能です。
以下の例をご覧ください。

In [ ]:
df2 = pd.read_csv(io.StringIO("""
x,y
1,2
3,4
"""))
df2

詳しく見ると、`pd.read_csv`はファイルのようなものを受け取ります、そして`io.StringIO`は文字からファイルのようなオブジェクトを作っています。

以下では、次のCSV形式のファイルを例に説明していきます。

In [ ]:
with open("data/tokyo-weather.csv") as f:
    [print(next(f), end='') for i in range(5)]

# データフレームとは (What is a data frame)


データフレームとは2次元の表形式のデータ(tabular data)を扱うためのデータ構造です。各列は型や名前がついています。列はそれぞれ型が異なってもよいです。
たとえば、スプレッドシートのデータはデータフレームとしてみることができます。

`pandas`のライブラリでの`DataFrame`クラスの定義はこちらを参考にしてください： https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html#pandas.DataFrame


例をみてみましょう。

In [ ]:
df = pd.read_csv('data/tokyo-weather.csv')
df.head(5)

##### `read_csv`はCSV形式のファイルからデータを読み込んでいます。

```python
# CSV形式のファイルからデータを読み込みます。
df = pd.read_csv('data/tokyo-weather.csv')
```

`df.head(5)`はデータの最初の５つの行を表示します。

```python
# 最初の５つの行を表示します。
df.head(5)
```

データフレームは値を含む「セル」が2次元格子状にならんだデータ構造です。
各セルには数値または文字列のデータを保存できます。
上の例ではいくつかのセルにNaN という値が入っていますが、これはNot A Numberの意味で、値が不正または欠けていることを表します

一つのデータフルームのよい考え方は統計分析に由来しますが、統計分析以外にもその考え方が役に立ちます。
各行は観測値を表し、各列は変数を表します。変数は直接に設定できる、または観測して図るものとします。
一つの観測値は同時に図るものなので、一つの行に入っている値は一つのものを記述します。
上記の例の表では、一つの行は一時間の観測を表しています。

Pythonの`pandas`のライブラリでは、列の型を知るために`dtypes`というプロパティを使用できます。
数値型は更に整数（`int64`)や浮動小数点(`float64`)の型に分けられます。文字の場合はオブジェクトの型（`object`)になります。

![dataframe](data/dataframe-ja.png)

In [ ]:
# データフレームの列の型をご覧できます。
# 因子はCSVの中で文字列として
df.dtypes

In [ ]:
# 一目で分かるデータの平均値や標準偏差
df.describe()

## キレイな(tidy)データフレーム (Tidy data frames)

データフレームにデータを入れる方法はたくさんありますが、それはどちらでもよいという訳はありません。以下の例を見ましょう。

    
| 日付        　 | 降水量      | 風向  |
| ------------- |------------|-------|
| 2019-08-08    | 50         | NE    |
| 2019-08-07    | 0          |   E   |

     
| 降水量.8/8  | 降水量.8/7  | 風向.8/8 |風向.8/7 |
| ------------- |---------|---------|---------|
|         50    | 0       | NE      |     E   | 

| 日付      | 変数      | 値    |
|----------|----------|-------|
|2019-08-08|降水量     |  50   |
|2019-08-08|風向       |  NE   |
|2019-08-07|降水量     |  0    |
|2019-08-07|風向      |  E    |

以上のデータの表現方法の中から一つは特に役に立ちます。それは「*キレイな(tidy)*データフレーム」といい、以下の条件に当てはまるデータフレームです。

* 一つのデータフレームに入るデータは一つの観測値として考えられ、変数は全て関連します。
* 一つの列は変数になります。列のヘッダは変数名です。変数の値はヘッダに絶対に入りません。
* ーつの行は一つの観測として考えられます。つまり、関係しないデータは一つの行に入りません。
  または、関連している観測した変数は一つの列に入れます。

キレイな(tidy)データフレームの条件に当てはまらないデータフレームは*汚い(messy)*といいます。
上の例では、１つ目の表はtidyで、２つ目と３つ目はmessyです。

データ解析の目的によって観測値の定義は異なる場合もあります。たとえば、飛行機の出発時間や到着時間は
別々の変数でしょうか。　飛行時間の解析であれば、別々の変数の扱いは便利です。なぜかというと、観測値ごとに
簡単に飛行時間を計算できるからです。　もし空港の飛行場の使い方の解析の場合は、離陸も着陸も飛行場を使う
機会なので、同じデータであっても、一つの変数にした方が解析しやすいのです。

詳しくキレイなデータフレームについてこちらの論文ご参考ください： https://vita.had.co.nz/papers/tidy-data.pdf （英語）

# 予習課題: 記述からデータフレームを生成 (Create data frame from textual description)


アリスはコーヒーを大好きで、よく飲みます。コーヒーの消費量に気になってデータ解析を行いたいので、以下の記述を読んで、データフレームをCSV形式で作ってください。
アリスの一週間の説明こちらです：

* アリスは平日は毎日に会社に通います。
* アリスは会社に着く前に毎日にコーヒーを飲みます。ただし、水曜日は飲みません。
* 平日の朝は、いつもSサイズのコップを買います。
* アリスは毎週火曜日と木曜日にジムに通います。
* ジムが終わったら、アリスはLサイズのコーヒーを飲んでいます。
* ジムがない日はコーヒー屋さんによらず直接に帰ります。
* 週末（土曜日と日曜日）は、アリスはコーヒーを家で一日一回作ります。一回の量は500mlです。
* Sサイズのコップは200ml, Lサイズのコップは300mlです。

課題として、データフレームを作って`coffee`という名前をつけてください。データフレームには以下の列を入れましょう。

* `day`: 整数、一週間の中の一日を記述します (1:月曜日, 2:火曜日, ..., 6:土曜日, 7:日曜日)
* `work`: 真理値、その日に会社に行くかどうか（1:会社に行く、0:行かない）
* `gym`: 真理値、その日にジムに行くかどうか（1:ジムに行く、0:行かない）
* `coffee_ml`: 整数、その日にコーヒーの消費量、mlの単位

In [ ]:
coffee = pd.read_csv(io.StringIO('''day,work,gym,coffee_ml
...
'''))

In [ ]:
# Inspect the resulting data frame
coffee

In [ ]:
# Test the data frame. **lang:ja**
assert len(coffee) == 7, "データフレームには７つの行が入らなければなりません"
assert 'day' in coffee, "データフレームには'day'の列が入らなければなりません"
assert 'coffee_ml' in coffee, "データフレームには'coffee_ml'の列が入らなければなりません"
assert 'work' in coffee, "データフレームには'work'の列が入らなければなりません"
assert 'gym' in coffee, "データフレームには'gym'の列が入らなければなりません"